In [16]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn import metrics
from pandas.tseries.offsets import *

In [17]:
# 注意要把表中的 0 也都按空值 nan 处理，因为实际上不可能有真正有意义的 0 出现的！
df = pd.read_csv("../../data/features/final_features_CB+OC.csv", dtype={'sale_date':str}, na_values=['-',0], low_memory=False)
df['sale_date']= pd.to_datetime(df['sale_date'])
# df = df.drop('C_rcm_0', axis=1)

In [18]:
# 单纯时间信息
df['year'] = df['sale_date'].apply(lambda x:x.year)
df['month'] = df['sale_date'].apply(lambda x:x.month)
df['is_leap_year'] = df['sale_date'].apply(lambda x:x.is_leap_year)
df['year_oh'] = df['year']
df['month_oh'] = df['month']
df = pd.get_dummies(df, columns=['year_oh','month_oh'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5727 entries, 0 to 5726
Columns: 1699 entries, class_id to month_oh_12
dtypes: bool(1), datetime64[ns](1), float64(1663), int64(16), uint8(18)
memory usage: 73.5 MB


In [19]:
# # 临时加一个把class_id做onehot的操作。
# df = pd.get_dummies(df, columns=['class_id'])

In [20]:
# %qtconsole
# pd.Series(df.columns.values).to_csv('all_columns.txt', index=False)

## 真正使用之前，还有最后一次机会，可以做特征的筛选！

In [21]:
# 最后的特征筛选！
# df = df[[blablabla]]
# df = df.drop([blabla], axis=1)

# drop = []
# for i in range(37, 60):
#     drop.append('C_som_' + str(i+1));
#     drop.append('C_fd_' + str(i+1));
#     drop.append('C_sd_' + str(i+1));
#     drop.append('C_fr_' + str(i+1));
#     drop.append('C_sr_' + str(i+1));
#     drop.append('C_dfr_' + str(i+1));
#     drop.append('C_rfd_' + str(i+1));
#     drop.append('C_rcm_' + str(i+1));
#     drop.append('C_rcm_fd_' + str(i+1));
#     drop.append('C_rcm_sd_' + str(i+1));
#     drop.append('C_rcm_fr_' + str(i+1));
#     drop.append('C_rcm_sr_' + str(i+1));
#     drop.append('C_rcm_dfr_' + str(i+1));
#     drop.append('C_rcm_rfd_' + str(i+1));

# df = df.drop(drop, axis=1)


In [22]:
df_backup = df

In [23]:
# 又得切一次，现在要切成三部分：2017.10以前的，2017.10的，2017.11的
# 第一步，得到验证集（2017.10）的 y
val_y = df[df['sale_date'] == pd.to_datetime('201710', format='%Y%m')].groupby('class_id').sum()['sale_quantity'].apply(lambda x: 20*x)#.apply(np.log2)

# 临时加一个把class_id做onehot的操作。
df = pd.get_dummies(df, columns=['class_id'])

# 第二步，得到训练集（2017.10以前）的 X 和 y
df_train = df[df['sale_date'] < pd.to_datetime('201710', format='%Y%m')]
# 第三步，得到验证集（2017.10）的 X
val_X = df[df['sale_date'] == pd.to_datetime('201710', format='%Y%m')]

In [24]:
val_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 31 to 5725
Columns: 1838 entries, sale_date to class_id_978089
dtypes: bool(1), datetime64[ns](1), float64(1663), int64(15), uint8(158)
memory usage: 1.8 MB


In [25]:
#处理训练集
# df_train = df
df_train = df_train.drop('sale_date', axis=1)
y = df_train['sale_quantity'].apply(lambda x: 20*x).values#.apply(np.log2).values
X = df_train.drop('sale_quantity', axis=1)

In [26]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5447 entries, 0 to 5724
Columns: 1836 entries, C_som_1 to class_id_978089
dtypes: bool(1), float64(1662), int64(15), uint8(158)
memory usage: 70.6 MB


In [27]:
#构造验证集
val_X = val_X.drop('sale_date', axis=1)
val_X = val_X.drop('sale_quantity', axis=1)


In [28]:
dtrain = xgb.DMatrix(X, label=y, missing=np.nan) 
# dtest = xgb.DMatrix(df_test)
dtest = xgb.DMatrix(val_X, label=val_y.values, missing=np.nan) 
param = {
    'max_depth' : 6,
    'eta' : 0.1, #0.02
    'objective' : 'reg:linear',
#     'objective':'binary:logistic',
    'silent': 0,
#     'nthread': 4,
#     'booster': 'gbtree'
}
num_round = 8000 # v0.2 best 129  # 8000
eval_set = [(dtrain, 'train'), (dtest, 'validation')]
best = xgb.train(param, dtrain, num_round, verbose_eval=True, early_stopping_rounds=20, evals=eval_set)#early stop 200

pred = best.predict(dtest).round(0)

[0]	train-rmse:13129.7	validation-rmse:10421.6
Multiple eval metrics have been passed: 'validation-rmse' will be used for early stopping.

Will train until validation-rmse hasn't improved in 20 rounds.
[1]	train-rmse:11972	validation-rmse:9391.53
[2]	train-rmse:10923.7	validation-rmse:8366.34
[3]	train-rmse:9977.6	validation-rmse:7530.51
[4]	train-rmse:9126.59	validation-rmse:6742.38
[5]	train-rmse:8365.61	validation-rmse:6086.21
[6]	train-rmse:7682.9	validation-rmse:5504.56
[7]	train-rmse:7070.67	validation-rmse:4993.52
[8]	train-rmse:6515.32	validation-rmse:4464.03
[9]	train-rmse:6017.21	validation-rmse:3991.09
[10]	train-rmse:5564.27	validation-rmse:3652.62
[11]	train-rmse:5161.38	validation-rmse:3303.27
[12]	train-rmse:4797.4	validation-rmse:3069.94
[13]	train-rmse:4479.57	validation-rmse:2823.46
[14]	train-rmse:4188.11	validation-rmse:2574.56
[15]	train-rmse:3928.92	validation-rmse:2382.23
[16]	train-rmse:3698.31	validation-rmse:2264.51
[17]	train-rmse:3490.96	validation-rmse:2158

In [29]:
# 测试指数对数处理labels
predSeries = pd.Series(pred)
ps = predSeries.values#.apply(lambda x: 2**x)
# ps = predSeries.apply(np.exp).values
val_y2metric = val_y.values

In [30]:
# 测试指数对数处理labels，计算最终结果
# np.sqrt( metrics.mean_squared_error(np.log(val_y.apply(lambda x: 2**x).values), np.log(ps)))
np.sqrt( metrics.mean_squared_error(np.log2(val_y2metric), np.log2(ps)))

0.55948553501277398

In [12]:
np.sqrt( metrics.mean_squared_error(val_y.values, pred))

119.86745656289344

In [13]:
%qtconsole
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(30, 250))
xgb.plot_importance(best, ax=ax)

### 看起来效果还不错！  
### 那就用完整的数据集训练一个结果交上去试试。  
### 迭代次数130，其他xgb参数同上。

先他妈的构造完整的训练集和测试集。
### 有部分特征需要重新生成！
#### 目前是**涉及车型**的统计信息，需要重新生成

### 生成特征的顺序是：
1. 环比同比之类的时间+车型信息，存在tmp中（5727条） —— 这部分特征是可以在后面预测11月时直接用的
2. 切分训练集、验证集/测试集出来
2. 车型自身统计信息，存在ndf（140条）
3. tmp.join(ndf)
4. 时间信息，直接加在上一步的结果上的

### 所以思路是：
1. 先重新读数据进来，和一开始一样
2. 切分训练集、测试集
3. 统计车型信息
4. tmp.join(ndf)
5. 加上时间信息


# 流程！
1. 车型历史销量特征，从文件读取（5727行）
2. 品牌历史销量特征，从文件读取（1600+行）
3. join
1. 最后是纯时间特征
-----
0. 备份全集于df_backup，因为下面要破坏，但后面还要用
1. 切分训练集、验证集
2. 训练并验证
3. 切分训练集、测试集
4. 测试集上跑出结果
------
+ 要么融合
+ 要么加高斯
+ 要么兼有

In [28]:
df = df_backup

In [20]:
# 又得切一次，现在要切成2部分：2017.11以前的，2017.11的

# # 第一步，得到验证集（2017.10）的 y
# val_y = df[df['sale_date'] == pd.to_datetime('201710', format='%Y%m')].groupby('class_id').sum()['sale_quantity']

# 第二步，得到训练集（2017.11以前）的 X 和 y
df_train_total = df[df['sale_date'] < pd.to_datetime('201711', format='%Y%m')]

# 第三步，得到测试集（2017.11）的 X
test_X = df[df['sale_date'] == pd.to_datetime('201711', format='%Y%m')]

In [21]:
# df_train_total = df
df_train_total = df_train_total.drop('sale_date', axis=1)
y_total = df_train_total['sale_quantity'].values
X_total = df_train_total.drop('sale_quantity', axis=1)

dtrain_total = xgb.DMatrix(X_total, label=y_total, missing=np.nan) 

test_X = test_X.drop('sale_date', axis=1)
test_X = test_X.drop('sale_quantity', axis=1)
dtest_total = xgb.DMatrix(test_X, missing=np.nan) 


In [23]:
param = {
    'max_depth' : 6,
    'eta' : 0.02,
    'objective' : 'reg:linear',
#     'objective':'binary:logistic',
    'silent': 0,
#     'nthread': 4,
#     'booster': 'gbtree'
}
num_round = 130

eval_set = [(dtrain_total, 'train')]
best = xgb.train(param, dtrain, num_round, verbose_eval=True, early_stopping_rounds=250, evals=eval_set)

pred_Nov = best.predict(dtest_total).round(0)

[0]	train-rmse:704.736
Will train until train-rmse hasn't improved in 250 rounds.
[1]	train-rmse:692.074
[2]	train-rmse:679.687
[3]	train-rmse:667.568
[4]	train-rmse:655.661
[5]	train-rmse:644.007
[6]	train-rmse:632.521
[7]	train-rmse:621.354
[8]	train-rmse:610.309
[9]	train-rmse:599.497
[10]	train-rmse:588.98
[11]	train-rmse:578.581
[12]	train-rmse:568.422
[13]	train-rmse:558.529
[14]	train-rmse:548.767
[15]	train-rmse:539.213
[16]	train-rmse:529.849
[17]	train-rmse:520.659
[18]	train-rmse:511.722
[19]	train-rmse:502.914
[20]	train-rmse:494.309
[21]	train-rmse:485.881
[22]	train-rmse:477.602
[23]	train-rmse:469.501
[24]	train-rmse:461.535
[25]	train-rmse:453.729
[26]	train-rmse:446.068
[27]	train-rmse:438.595
[28]	train-rmse:431.283
[29]	train-rmse:424.118
[30]	train-rmse:417.086
[31]	train-rmse:410.16
[32]	train-rmse:403.412
[33]	train-rmse:396.811
[34]	train-rmse:390.345
[35]	train-rmse:383.98
[36]	train-rmse:377.767
[37]	train-rmse:371.683
[38]	train-rmse:365.745
[39]	train-rmse:35

In [24]:
res = pd.read_csv('../../data/origin/yancheng_testA_20171225.csv')
res['predict_quantity'] = pred_Nov
res.to_csv("../../result/v0.5.csv",index=False)

### 简单ensemble一波

In [20]:
res1 = pd.read_csv('../../result/Oct_mul_115percents.csv')
pred1 = res1['predict_quantity']

res2 = pd.read_csv('../../result/v0.3.csv')
pred2 = res2['predict_quantity']

final_pred = pd.Series(0.6*pred1 +  0.4*pred2).round(0).values


res = pd.read_csv('../../data/origin/yancheng_testA_20171225.csv')
res['predict_quantity'] = final_pred
res.to_csv("../../result/Ensembled_0.6oct+0.4v0.3.csv",index=False)

用十一月的好结果乘以1.4，作为十二月的。

In [8]:
dec = pd.read_csv('../../data/origin/yancheng_testB_20180224.csv')
dec

,predict_date,class_id,predict_quantity
0,201712,281792,NaN
1,201712,682651,NaN
2,201712,603972,NaN
3,201712,221795,NaN
4,201712,482497,NaN
5,201712,379265,NaN
6,201712,687270,NaN
7,201712,890189,NaN
8,201712,654134,NaN
9,201712,739296,NaN


In [9]:
nov_pred = pd.read_csv('../../result/20180202_half_ensemble.csv')
nov_pred['tmp'] = nov_pred['predict_quantity'] * 1.4
nov_pred

,predict_date,class_id,predict_quantity,tmp
0,201711,103507,206.0,288.4
1,201711,124140,279.0,390.6
2,201711,125403,194.0,271.6
3,201711,136916,177.0,247.8
4,201711,169673,168.0,235.2
5,201711,175962,250.0,350.0
6,201711,178529,183.0,256.2
7,201711,186250,109.0,152.6
8,201711,194201,396.0,554.4
9,201711,194450,293.0,410.2


In [10]:
dec = pd.merge(dec, nov_pred[['class_id','tmp']], how='left', on='class_id')
dec['predict_quantity'] = dec['tmp'].round(0)
dec

,predict_date,class_id,predict_quantity,tmp
0,201712,281792,658.0,658.0
1,201712,682651,449.0,449.4
2,201712,603972,647.0,646.8
3,201712,221795,568.0,568.4
4,201712,482497,286.0,285.6
5,201712,379265,480.0,480.2
6,201712,687270,659.0,659.4
7,201712,890189,1901.0,1901.2
8,201712,654134,895.0,894.6
9,201712,739296,1260.0,1260.0


In [12]:
dec = dec.drop('tmp',1)

In [13]:
dec

,predict_date,class_id,predict_quantity
0,201712,281792,658.0
1,201712,682651,449.0
2,201712,603972,647.0
3,201712,221795,568.0
4,201712,482497,286.0
5,201712,379265,480.0
6,201712,687270,659.0
7,201712,890189,1901.0
8,201712,654134,895.0
9,201712,739296,1260.0


In [14]:
dec.to_csv("../../result/Dec_NovPred_multiBy1.4.csv",index=False)